# modified Berman Standard State Code Generator 
Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
sym.init_printing()

Required ENKI packages

In [ ]:
from thermoengine import coder

# Types of terms in a standard state properties description
There are three classes of terms:
1. Terms that apply over the whole of $T$-, $P$-space, $T_r \le T$, $P_r \le P$
2. Terms that apply over a specified range of $T$-, $P$-space, $(T_{r_\lambda},P_{r_\lambda}) \le (T,P) \le (T_\lambda,P_\lambda)$
3. Terms that apply to a specific $T_t$ and $P_t$ and higher $T$, $P$, $T_t \le T$, $P_t \le P$

Second-order phase transitions ($lambda$-transitions) are an example of the second type, as are order disorder transformations. First-order phase transitions are an example of the third type.  

## Create a model class for the Gibbs free energy

In [ ]:
model = coder.StdStateModel()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

### Define model expressions applicable over all of T,P space
An expression for the Gibbs free energy, $G(T,P)$ or the Helmholtz energy $A(T,V)$ is constructed.  The expression may have multiple parts.  Often the heat capacity function is postulated, then integrated to yield expressions for the entahlpy, entropy, and in combination the energy potential. Then, an equation of state (EOS) is adopted and that term is integrated in pressure or volume and added to the heat capacity integrals. This proceedure is follwed here.
#### (1) $C_P$ integrals
The isobaric heat capacity terms parameterized as: $C_P =  $,
and in addition the reference condition third law entropy, $ S_{Tr,Pr} $, and enthalpy of formation from the
elements, $ \Delta H_{Tr,Pr} $, constitute additional parameters:

* NOTE this does not include critical heat capacity contribution

In [ ]:
# a_cp,b_cp,c_cp,d_cp= sym.symbols('a_cp b_cp c_cp d_cp')
# S_max = sym.symbols('S_max')
# CpPr = a_cp+b_cp*T+c_cp/T**2+d_cp/sym.sqrt(T)
# STrPr,HTrPr = sym.symbols('S_TrPr H_TrPr')
# CpPr

In [ ]:
k0,k1,k2,k3 = sym.symbols('k0 k1 k2 k3')
CpPr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3
STrPr,HTrPr = sym.symbols('S_TrPr H_TrPr')

Specify parameters ...

In [ ]:
params = [('H_TrPr','J',HTrPr), ('S_TrPr','J/K',STrPr), ('k0','J/K-m',k0), ('k1','J-K^(1/2)-m',k1),
          ('k2','J-K/m',k2),  ('k3','J-K^2',k3)]

Define the heat capacity contribution to the Gibbs free energy ...

In [ ]:
GPr = HTrPr + sym.integrate(CpPr,(T,Tr,T)) - T*(STrPr + sym.integrate(CpPr/T,(T,Tr,T)))

In [ ]:
GPr

... and add this expression to the model

In [ ]:
model.add_expression_to_model(GPr, params)

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space

In [ ]:

VTrPr,a0,K0,K_P,K_P2 = sym.symbols('V_TrPr a0 K0 K_P K_P2')
a_v,b_v,c_v = sym.symbols('a_v b_v c_v')

Pth = sym.symbols('P_th')
# V_T, K_T, dK_T, dK_P = sym.symbols("V_T K_T K'_T K'_P")
# dK_P 

In [ ]:
V0T,K0T = sym.symbols('V_0T K_0T')

In [ ]:
# V = V0T*( 1 - a_v*(1-(1+b_v*P)**(-c_v)) )
# V

In [ ]:
# V = V0T*( 1 - sym.sqrt((1+c_v)/c_v)*(1-(1+b_v*P)**(-c_v)) )
# V

In [ ]:
V = V0T*( 1 + K_P/K0T*P)**(-1/K_P)
V

In [ ]:
# V = VTrPr*( 1 + K_P/K0*P)**(-1/K_P)
# V

In [ ]:
VdP = sym.integrate(V, (P, 0, P), conds='none')
VdP = VdP.simplify()

# VdP = VdP.subs(a_v, (1+K_P)/(1+K_P+K0T*K_P2))
VdP

In [ ]:
# # V = V.subs(a_v, (1+K_P)/(1+K_P+K0T*K_P2))
# V = V.subs(b_v, K_P/K0T - K_P2/(1+K_P))
# V = V.subs(c_v, (1+K_P+K0T*K_P2)/(K_P**2 +K_P - K0T*K_P2))
# V = V.subs(K_P2, -K_P/K0)
# V

In [ ]:
# VdP = sym.integrate(V, (P, 0, P), conds='none')
# VdP = VdP.simplify()
# 
# # VdP = VdP.subs(a_v, (1+K_P)/(1+K_P+K0T*K_P2))
# VdP = VdP.subs(b_v, K_P/K0T - K_P2/(1+K_P))
# VdP = VdP.subs(c_v, (1+K_P+K0T*K_P2)/(K_P**2 +K_P - K0T*K_P2))
# VdP = VdP.subs(K_P2, -K_P/K0)
# VdP.simplify()

In [ ]:
alpha0 = sym.symbols('alpha0')

V0T_expr = VTrPr*(1+alpha0*(T-Tr))
V0T_expr

In [ ]:
K0T_expr = K0*(VTrPr/V0T_expr)**K_P
K0T_expr

In [ ]:
V = V.subs(V0T, V0T_expr)
V = V.subs(K0T, K0T_expr)
V

In [ ]:
VdP = VdP.subs(V0T, V0T_expr)
VdP = VdP.subs(K0T, K0T_expr)
VdP.simplify()

In [ ]:
VdP.free_symbols

In [ ]:
# params= [('V_TrPr', 'J/bar-m', VTrPr), ('a0','1/K',a0),  ('K0','bar',K0), ('dKdT','1/K',dK_T)]
params= [('V_TrPr', 'J/bar-m', VTrPr), 
         ('alpha0','1/K',alpha0),  ('K0','bar',K0),  ('K_P','',K_P)]

In [ ]:
model.add_expression_to_model(VdP, params)